# Decision Making Under Uncertainty - Assignment 2

Group 2:
- Martijn Ketelaars (ANR: 120975)
- Robbie Reyerse (ANR: 109997)
- Rosalien Timmerhuis (ANR: 520618)
- Mike Weltevrede (ANR: 756479)

## Exercise a.
Derive the product-form solution for the stationary distribution of this Jackson network and determine the stability conditions (consult Lecture on 30 October).

## Exercise b.
Write for this extended system a discrete-event simulation. In order to do this, you might need object-oriented programming.

In [ ]:
from scipy import stats
from dist.Distribution import Distribution

import heapq

from numpy.ma.core import zeros
import matplotlib.pyplot as plt

In [1]:
# Suppose one customer
# Remember the priorities!

class Customer :

    def __init__(self, arrivalTime, serviceTime, priority):
        self.arrivalTime = arrivalTime
        self.serviceTime = serviceTime
        self.systemArrivalTime = arrivalTime
        self.priority = priority
        
        # location: holding, needy, content, out
        self.location = 'holding'
        
    def moveTo(self, location, time, newServiceTime):
        self.location = location
        self.arrivalTime = time
        self.serviceTime = newServiceTime
        
    def leaveSystem(self):
        self.location = 'out'
        self.serviceTime = -1
        

In [2]:
class Event:

    ARRIVAL = 0
    DEPARTURE = 1
    
    def __init__(self, typ, time, cust): 
        self.type = typ
        self.time = time
        self.customer = cust
        
    def __str__(self):
        s = ('Arrival', 'Departure')
        return s[self.type] + " of customer " + str(self.customer)
        + ' at t = ' + str(self.time)


In [ ]:
class FES :
    
    def __init__(self):
        self.events = []
        
    def add(self, event):
        heapq.heappush(self.events, event)
        
    def next(self):
        return heapq.heappop(self.events)
    
    def isEmpty(self):
        return len(self.events) == 0
        
    def __str__(self):
        # Note that if you print self.events, it would not appear to be sorted
        # (although they are sorted internally).
        # For this reason we use the function 'sorted'
        s = ''
        sortedEvents = sorted(self.events)
        for e in sortedEvents :
            s += str(e) + '\n'
        return s


In [3]:
class SimResults:
    
    MAX_QL = 10000
    
    def __init__(self):
        self.sumQL = 0
        self.oldTime = 0
        self.queueLengthHistogram = zeros(self.MAX_QL + 1)
        
    def registerQueueLength(self, time, ql):
        self.sumQL += ql * (time - self.oldTime)
        self.queueLengthHistogram[min(ql, self.MAX_QL)] += (time - self.oldTime)
        self.oldTime = time
        
    def getMeanQueueLength(self): 
        return self.sumQL / self.oldTime
    
    def getQueueLengthProbabilities(self) :
        return [x/self.oldTime for x in self.queueLengthHistogram]

    def plotQueueLengthHistogram(self, maxq=50):
        ql = self.getQueueLengthHistogram()
        maxx = maxq + 1
        plt.figure()
        plt.bar(range(0, maxx), ql[0:maxx])
        plt.ylabel('P(Q = k)')
        plt.xlabel('k')
        plt.show()
    
    def __str__(self):
        ql = self.getQueueLengthProbabilities()
        s = 'Mean queue length: ' + str(self.getMeanQueueLength()) + '\n'
        s += 'Queue-length probabilities: \n'
        for i in range(21):
            s += 'P(Q = ' + str(i) + ') = ' + str(ql[i]) + '\n'
        return s
    

In [ ]:
class GGcSimulationQL :
    
    def __init__(self, arrDist, servDist, nrServers=1):   
        self.arrDist = arrDist
        self.servDist = servDist
        self.nrServers = nrServers
        
    def simulate(self, T):
        q = 0  # current number of customers
        t = 0  # current time
        res = SimResults()
        fes = FES()
        firstEvent = Event(Event.ARRIVAL, self.arrDist.rvs())  
        fes.add(firstEvent)                                 
        while t < T :
            e = fes.next()                      # jump to next event
            t = e.time                          # update the time
            res.registerQueueLength(t, q)       # register the queue length
            if e.type == Event.ARRIVAL :        # if event is an arrival:    
                q += 1                          # increase queue length
                if q <= self.nrServers :     # there is an available server
                    dep = Event(Event.DEPARTURE, t + self.servDist.rvs())
                    fes.add(dep)
                arr = Event(Event.ARRIVAL, t + self.arrDist.rvs()) 
                fes.add(arr)                    # schedule next arrival
            elif e.type == Event.DEPARTURE :    # event is departure  
                q -= 1                          # decrease queue length
                if q >= self.nrServers :     # someone is waiting for service
                    dep = Event(Event.DEPARTURE, t + self.servDist.rvs())
                    fes.add(dep)
        return res
    

In [ ]:
arrDist = Distribution(stats.expon(scale=1/2.0))
servDist = Distribution(stats.expon(scale=1/1.0))
sim = GGcSimulationQL(arrDist, servDist, 3)
res = sim.simulate(1000000)
print(res)

## Exercise c.
Use your discrete-event simulation to present performance measures (perhaps probability of blocking for the different customer priority classes, probability of waiting, mean queue length, mean waiting time,...) for the extended system for different values of $\lambda_L$, $\lambda_H$, and $\delta$, and different values of $N$. Elaborate on your findings by creating a comprehensive management report including several sensitivity analyses.

## Exercise d.
Investigate the didderence in performance between the original system ($N = 1$) and the extended system (finite $N$). For instance, how fast does the behavior of the extended system mimic the original system behavior as $N$ increases?